# Disruptive Papers in Fetal Surgery

## Description

This notebook takes as input one or more PubMed CSV exports, placed in the [data/pubmed/results/](data/pubmed/results/) directory. It adds useful columns to the CSV file, maps the PubMed IDs to Microsoft Academic Graph IDs, uses the MAGIDs to look up citation and disruption data from Wu et al., 2023 to score each article, and saves the scored output to a styled Excel spreadsheet.

### PubMed Search Documentation and Replication

The searches used to create the PubMed CSV exports are documented in [DisruptivePapersFetalSurgery-PubMedSearchDocumentation-202303.xlsx](data/pubmed/searches/DisruptivePapersFetalSurgery-PubMedSearchDocumentation-202303.xlsx). They cover Congenital Diaphragmatic Hernia (CDH), Congenital Pulmonary Airway Malfunction (CPAM), Neural Tube Defects (NTD), and Twin-to-Twin Transfusion Syndrome (TTTS), with each search documented on a separate spreadsheet tab. 

The searches can be replicated by either copying and pasting the full search string from each spreadsheet tab into PubMed or clicking on the number in the Results column. Once replicated, the search results can be exported to CSV via the *Save* button. Select *All results* and *CSV* format, then click *Create file*.

### References

Wu, L., Wang, D., & Evans, J. (2023). Replication Data for: Large teams develop and small teams disrupt science and technology [Data set]. Harvard Dataverse. https://doi.org/10.7910/DVN/JPWNNK


## Environment

### Import libraries

In [ ]:
from collections import Counter
import glob
import logging
import pandas as pd
from pathlib import Path
import requests
from styleframe import StyleFrame, Styler, utils
import time
from tqdm.notebook import tqdm

### Define constants

In [ ]:
email = "whimar@ohsu.edu"
openalex = "https://api.openalex.org/works?per-page=100&filter=pmid:"
headers = {"user-agent": "mailto:" + email}

datestamp = time.strftime("%Y%m%d")
TOP_PAPERS = 500

DATA_DIR = "data/"
MAGDISRUPT_DIR = DATA_DIR + "MAG-disruption/"
# MAGDISRUPT_FILE = MAGDISRUPT_DIR + "AggregatedMAG.txt"
MAGDISRUPT_FILE = MAGDISRUPT_DIR + "Aggregated_20210521.txt"
PUBMED_DIR = DATA_DIR + "pubmed/"
RESULTS_DIR = PUBMED_DIR + "results/*"

LOG_DIR = "log/"

TARGET_DIR = "target/"
CSV_DIR = TARGET_DIR + "csv/"
XLSX_DIR = TARGET_DIR + "xlsx/"

### Create target directories

In [ ]:
Path(LOG_DIR).mkdir(parents=True, exist_ok=True)
Path(TARGET_DIR).mkdir(parents=True, exist_ok=True)
Path(CSV_DIR).mkdir(parents=True, exist_ok=True)
Path(XLSX_DIR).mkdir(parents=True, exist_ok=True)

### Configure logging

In [ ]:
logging.basicConfig(
    filename=f"{LOG_DIR}fsdp-{datestamp}.log",
    filemode="w",
    force=True,
    format="%(asctime)s.%(msecs)03d : %(levelname)s : %(message)s",
    level=logging.INFO,
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger()

### Load disruption data

In [ ]:
logger.info("Reading disruption data from %s", MAGDISRUPT_FILE)
# AggregatedMAG
# MAGDATA_DF = pd.read_csv(
#     MAGDISRUPT_FILE,
#     sep="\t",
#     usecols=[0, 5, 6],
#     names=["magid", "num_citations", "disruption_score"],
#     index_col="magid",
#     dtype={"num_citations": "int64", "disruption_score": "float64"},
# )

# Aggregated_20210521
MAGDATA_DF = pd.read_csv(
    MAGDISRUPT_FILE,
    sep="\t",
    usecols=[0, 4, 5],
    names=["magid", "num_citations", "disruption_score"],
    index_col="magid",
    dtype={"num_citations": "int64", "disruption_score": "float64"},
)
MAGDATA_DF.sort_index(inplace=True)

## Function definitions

### pmid2magid



#### Description

Maps PubMed IDs (PMID) from a PubMed CSV export to Microsoft Academic Graph IDs (MAGID) using the [OpenAlex Works API](https://docs.openalex.org/api-entities/works). Caches API results to CSV and reads from CSV if it already exists to reduce API calls.

#### Parameters

| Parameter | Description |
| --- | --- |
| articles_df | A processed/augmented DataFrame created from a PubMed CSV export |
| topicdate | The stem of the PubMed CSV filename, expected filename convenion *topic-date.csv* |

#### Return Values

| Return Value | Description |
| --- | --- |
| mags_df | a two-column dataframe of PMIDs and MAGIDs |


In [ ]:
def pmid2magid(articles_df, topicdate):
    magsfile = f"{CSV_DIR}{topicdate}-PMID2MAGID.csv"
    magsfilepath = Path(magsfile)

    if magsfilepath.is_file():
        logger.info("%s : Reading cached MAGID data from file %s", topicdate, magsfile)
        mags_df = pd.read_csv(magsfile, index_col="long_pmid")
    else:
        logger.info("%s : Pulling new MAGID data from OpenAlex", topicdate)
        pmids = articles_df["pmid"].tolist()

        # combine PMIDs into groups of 50 to reduce OpenAlex API calls
        pmid_groups = [pmids[i : i + 50] for i in range(0, len(pmids), 50)]

        pmid_strings = []
        mags = {}
        for pmid_group in pmid_groups:
            pmid_strings.append("|".join(map(str, pmid_group)))

        for pmid_string in tqdm(pmid_strings, desc=topicdate + " openalex"):
            logger.info("%s : Requesting PMIDs from OpenAlex: %s", topicdate, pmid_string)
            response = requests.get(openalex + pmid_string, headers=headers)
            logger.info("%s : OpenAlex response code: %s", topicdate, response.status_code)
            if response.status_code == 200:
                works = response.json()
                for work in works["results"]:
                    if "mag" in work["ids"]:
                        logger.info(
                            "%s : Found MAGID %s for PMID %s", topicdate, work["ids"]["mag"], work["ids"]["pmid"]
                        )
                        mags[work["ids"]["pmid"]] = work["ids"]["mag"]
                    else:
                        logger.info("%s : No MAGID found for PMID %s", topicdate, work["ids"]["pmid"])
            else:
                logger.error("%s : Error making OpenAlex request for %s", topicdate, pmid_string)

        mags_df = pd.DataFrame.from_dict(mags, orient="index", columns=["magid"])
        mags_df.index.name = "long_pmid"
        mags_df.sort_index(inplace=True)
        num_magid = len(mags_df.index)
        num_pmid = len(articles_df.index)
        logger.info("%s : Found %d MAGIDs from %d PMIDs", topicdate, num_magid, num_pmid)
        logger.info("%s : %d MAGIDs not found", topicdate, num_pmid - num_magid)
        logger.info("%s : Saving mapped PMID/MAGIDs to %s", topicdate, magsfile)
        mags_df.to_csv(magsfile)
        mags_df = pd.read_csv(magsfile, index_col="long_pmid")

    return mags_df

### score_results


#### Description

Maps PubMed IDs (PMID) from a PubMed CSV export to citation and disruption score data.

#### Parameters

| Parameter | Description |
| --- | --- |
| articles_df | A processed/augmented DataFrame created from a PubMed CSV export |
| topicdate | The stem of the PubMed CSV filename, expected format topic-date.csv |

#### Return Values

| Return Value | Description |
| --- | --- |
| cited_df | A processed/augmented DataFrame created from a PubMed CSV export and sorted by descending number of citations |
| develop_df | A processed/augmented DataFrame created from a PubMed CSV export and sorted from most developmental to least developmental |
| disrupt_df |A processed/augmented DataFrame created from a PubMed CSV export and sorted from most disruptive to least disruptive |


In [ ]:
def score_results(articles_df, topicdate):
    citefile = f"{CSV_DIR}{topicdate}-topcited.csv"
    developfile = f"{CSV_DIR}{topicdate}-topdevelopmental.csv"
    disruptfile = f"{CSV_DIR}{topicdate}-topdisruptive.csv"

    logger.info("%s : Joining article data with disruption data", topicdate)
    scored_df = articles_df.join(MAGDATA_DF, sort=True)
    scored_df.reset_index(inplace=True)

    logger.info("%s : Sorting articles by descending citation count", topicdate)
    cited_df = scored_df[
        [
            "num_citations",
            "title",
            "journal",
            "pubdate",
            "magid",
            "pmid",
            "doi",
            "pubmed",
            "ohsu_library",
            "rush_library",
        ]
    ].nlargest(TOP_PAPERS, columns="num_citations")

    logger.info("%s : Saving citation count data to %s", topicdate, citefile)
    cited_df.to_csv(
        citefile,
        index=False,
        columns=[
            "num_citations",
            "magid",
            "pmid",
            "title",
            "journal",
            "pubdate",
            "doi",
        ],
    )

    logger.info("%s : Selecting most developmental articles", topicdate)
    development_df = scored_df[
        [
            "disruption_score",
            "title",
            "journal",
            "pubdate",
            "magid",
            "pmid",
            "doi",
            "pubmed",
            "ohsu_library",
            "rush_library",
        ]
    ].nsmallest(TOP_PAPERS, columns="disruption_score")

    logger.info("%s : Saving developmental articles data to %s", topicdate, developfile)
    development_df.to_csv(
        developfile,
        index=False,
        columns=[
            "disruption_score",
            "magid",
            "pmid",
            "title",
            "journal",
            "pubdate",
            "doi",
        ],
    )

    logger.info("%s : Selecting most disruptive articles", topicdate)
    disrupt_sf = scored_df[
        [
            "disruption_score",
            "title",
            "journal",
            "pubdate",
            "magid",
            "pmid",
            "doi",
            "pubmed",
            "ohsu_library",
            "rush_library",
        ]
    ].nlargest(TOP_PAPERS, columns="disruption_score")

    logger.info("%s : Saving disruptive articles to %s", topicdate, disruptfile)
    disrupt_sf.to_csv(
        disruptfile,
        index=False,
        columns=[
            "disruption_score",
            "magid",
            "pmid",
            "title",
            "journal",
            "pubdate",
            "doi",
        ],
    )

    return cited_df, development_df, disrupt_sf

### style_output


#### Description

Exports sorted scored dataframes to styled XLSX file.

#### Parameters

| Parameter | Description |
| --- | --- |
| cited_df | A processed/augmented DataFrame created from a PubMed CSV export and sorted by descending number of citations |
| develop_df | A processed/augmented DataFrame created from a PubMed CSV export and sorted from most developmental to least developmental |
| disrupt_df |A processed/augmented DataFrame created from a PubMed CSV export and sorted from most disruptive to least disruptive |
| topicdate | The stem of the PubMed CSV filename, expected format topic-date.csv |

#### Return Values

None.

In [ ]:
def style_output(cited_df, develop_df, disrupt_sf, topicdate):
    scorefile = f"{XLSX_DIR}{topicdate}-scores.xlsx"

    cited_sf = StyleFrame(cited_df.drop("pmid", axis=1))
    develop_sf = StyleFrame(develop_df.drop("pmid", axis=1))
    disrupt_sf = StyleFrame(disrupt_sf.drop("pmid", axis=1))
    # query_s = pd.Series({f"{querytype} query": query})

    default_style = Styler(
        font=utils.fonts.calibri,
        font_size=11,
        border_type=utils.borders.default_grid,
        horizontal_alignment=utils.horizontal_alignments.left,
        wrap_text=False,
        shrink_to_fit=False,
    )

    header_style = Styler(
        bg_color=utils.colors.black,
        bold=True,
        font=utils.fonts.calibri,
        font_color=utils.colors.white,
        font_size=14,
        horizontal_alignment=utils.horizontal_alignments.left,
        shrink_to_fit=False,
        wrap_text=False,
        vertical_alignment=utils.vertical_alignments.center,
    )
    hyperlink_style = Styler(
        font_color=utils.colors.blue,
        protection=True,
        underline=utils.underline.single,
    )
    float_style = Styler(
        number_format="0.000000000000",
        horizontal_alignment=utils.horizontal_alignments.right,
    )

    cited_sf.set_column_width_dict(
        col_width_dict={
            ("pubdate", "magid", "pubmed"): 12,
            ("num_citations", "ohsu_library", "rush_library"): 15,
            ("journal"): 30,
            ("title", "doi"): 50,
        }
    )
    cited_sf.apply_headers_style(header_style)
    cited_sf.apply_column_style(cited_sf.columns, styler_obj=default_style)
    cited_sf.apply_column_style(
        ["doi", "pubmed", "ohsu_library", "rush_library"],
        styler_obj=Styler.combine(default_style, hyperlink_style),
    )

    develop_sf.set_column_width_dict(
        col_width_dict={
            ("pubdate", "magid", "pubmed"): 12,
            ("ohsu_library", "rush_library"): 15,
            ("disruption_score"): 20,
            ("journal"): 30,
            ("title", "doi"): 50,
        }
    )
    develop_sf.apply_headers_style(header_style)
    develop_sf.apply_column_style(develop_sf.columns, styler_obj=default_style)
    develop_sf.apply_column_style("disruption_score", styler_obj=Styler.combine(default_style, float_style))
    develop_sf.apply_column_style(
        ["doi", "pubmed", "ohsu_library", "rush_library"],
        styler_obj=Styler.combine(default_style, hyperlink_style),
    )

    disrupt_sf.set_column_width_dict(
        col_width_dict={
            ("pubdate", "magid", "pubmed"): 12,
            ("ohsu_library", "rush_library"): 15,
            ("disruption_score"): 20,
            ("journal"): 30,
            ("title", "doi"): 50,
        }
    )
    disrupt_sf.apply_headers_style(header_style)
    disrupt_sf.apply_column_style(disrupt_sf.columns, styler_obj=default_style)
    disrupt_sf.apply_column_style("disruption_score", styler_obj=Styler.combine(default_style, float_style))
    disrupt_sf.apply_column_style(
        ["doi", "pubmed", "ohsu_library", "rush_library"],
        styler_obj=Styler.combine(default_style, hyperlink_style),
    )

    with pd.ExcelWriter(scorefile) as sfile:
        logger.info("%s : Saving styled citation count data to %s", topicdate, scorefile)
        cited_sf.to_excel(
            sfile,
            index=False,
            columns=[
                "num_citations",
                "title",
                "journal",
                "pubdate",
                "magid",
                "doi",
                "pubmed",
                "ohsu_library",
                "rush_library",
            ],
            sheet_name=f"top {TOP_PAPERS} cited",
        )
        logger.info("%s : Saving styled developmental score data to %s", topicdate, scorefile)
        develop_sf.to_excel(
            sfile,
            index=False,
            columns=[
                "disruption_score",
                "title",
                "journal",
                "pubdate",
                "magid",
                "doi",
                "pubmed",
                "ohsu_library",
                "rush_library",
            ],
            sheet_name=f"top {TOP_PAPERS} developmental",
        )

        logger.info("%s : Saving styled disruption score data to %s", topicdate, scorefile)
        disrupt_sf.to_excel(
            sfile,
            index=False,
            columns=[
                "disruption_score",
                "title",
                "journal",
                "pubdate",
                "magid",
                "doi",
                "pubmed",
                "ohsu_library",
                "rush_library",
            ],
            sheet_name=f"top {TOP_PAPERS} disruptive",
        )
        # query_s.to_excel(sfile, index=False, header=False, sheet_name="pubmed query")

# Read and Augment PubMed CSV for MAGID mapping and citation/disruption scoring

In [ ]:
pubmedCSVs = sorted(glob.glob(RESULTS_DIR))

for pubmedCSV in pubmedCSVs:
    topicdate = Path(pubmedCSV).stem

    logger.info("%s : Reading search results from PubMed CSV %s", topicdate, pubmedCSV)

    pmresults_df = pd.read_csv(
        pubmedCSV, header=0, usecols=[0, 1, 5, 6, 10], names=["pmid", "title", "journal", "pubdate", "doi"]
    )

    # add columns of helpful URLs
    # hyperlinked URLs makes publications more accessible in the XLSX output

    # DOI link, blank if no DOI
    pmresults_df["doi"] = [
        f'=HYPERLINK("https://doi.org/{doi}")' if pd.notna(doi) else "" for doi in pmresults_df["doi"]
    ]

    # Link to PubMed record
    pmresults_df["pubmed"] = [
        f'=HYPERLINK("https://pubmed.ncbi.nlm.nih.gov/{pmid}", {pmid})' for pmid in pmresults_df["pmid"]
    ]

    # Link to OHSU Library Catalog lookup for full-text access
    pmresults_df["ohsu_library"] = [
        f'=HYPERLINK("https://librarysearch.ohsu.edu/openurl/OHSU/OHSU?sid=Entrez:PubMed&id=pmid:{pmid}", "Find @ OHSU")'
        for pmid in pmresults_df["pmid"]
    ]

    # Link to Rush University Library Catalog lookup for full-text access
    pmresults_df["rush_library"] = [
        f'=HYPERLINK("https://i-share-rsh.primo.exlibrisgroup.com/openurl/01CARLI_RSH/01CARLI_RSH:CARLI_RSH?sid=Entrez:PubMed&id=pmid:{pmid}", "Find @ Rush")'
        for pmid in pmresults_df["pmid"]
    ]

    # "Long PMID" for MAGID mapping in OpenAlex
    pmresults_df["long_pmid"] = [f"https://pubmed.ncbi.nlm.nih.gov/{pmid}" for pmid in pmresults_df["pmid"]]

    pmresults_df.set_index("long_pmid", inplace=True)
    pmresults_df.sort_index(inplace=True)
    pmresults_df.to_csv(f"{CSV_DIR}{topicdate}-longPMID.csv")
    pmresults_df = pd.read_csv(f"{CSV_DIR}{topicdate}-longPMID.csv", index_col="long_pmid")

    logger.info("%s : Mapping PMIDs to MAGIDs", topicdate)
    mags_df = pmid2magid(pmresults_df, topicdate)

    logger.info("%s : Joining PubMed search results with MAGID mapping", topicdate)
    articles_df = mags_df.join(pmresults_df).set_index("magid").sort_index()
    articles_df.index = articles_df.index.astype("int64")
    articles_df.to_csv(f"{CSV_DIR}{topicdate}-MAGID.csv")
    articles_df = pd.read_csv(f"{CSV_DIR}{topicdate}-MAGID.csv", index_col="magid")

    logger.info("%s : Scoring articles", topicdate)
    articles_cited_df, articles_develop_df, articles_disrupt_df = score_results(articles_df, topicdate)

    logger.info("%s : Styling output XSLX", topicdate)
    style_output(articles_cited_df, articles_develop_df, articles_disrupt_df, topicdate)

In [ ]:
# Initialize a Counter to store occurrences
journal_counter = Counter()

# Initialize a dictionary to store aggregated search sources
search_source_aggregated = {}

# Iterate through each CSV file in the results directory
for pubmedCSV in pubmedCSVs:
    topicdate = Path(pubmedCSV).stem

    logger.info("%s : Reading search results from PubMed CSV %s", topicdate, pubmedCSV)

    pmresults_df = pd.read_csv(
        pubmedCSV, header=0, usecols=[0, 1, 5, 6, 10], names=["pmid", "title", "journal", "pubdate", "doi"]
    )
    # Add a column "search_source" containing the value of pubmedCSV
    pmresults_df["search_source"] = Path(pubmedCSV).stem

    # Update the counter with the values from the "journal" column
    journal_counter.update(pmresults_df["journal"].dropna())

    # Aggregate search_source for each journal
    for journal, sources in pmresults_df.groupby("journal")["search_source"]:
        if journal not in search_source_aggregated:
            search_source_aggregated[journal] = set()
        search_source_aggregated[journal].update(sources)

# Sort the counter by occurrences in descending order
sorted_journal_counts = journal_counter.most_common()

# Prepare the final dataframe
journal_counts_data = []
for journal, count in sorted_journal_counts:
    aggregated_sources = "|".join(sorted(search_source_aggregated[journal]))
    journal_counts_data.append((journal, count, aggregated_sources))

journal_counts_df = pd.DataFrame(journal_counts_data, columns=["Journal", "Count", "Search_Source"])

# Export the journal counter to a CSV file
journal_counts_df.to_csv(f"{CSV_DIR}search_journal_counts.csv", index=False)

# Display the final dataframe
journal_counts_df

In [ ]:
scoredCSVs = sorted(glob.glob(f"{CSV_DIR}*top*.csv"))
scoredCSVs

In [ ]:
from collections import Counter


# Initialize a Counter to store occurrences
pubmed_pubmed_pubmed_pubmed_pubmed_journal_counter = Counter()

# Initialize a dictionary to store aggregated search sources
search_source_aggregated = {}

# Iterate through each CSV file in the results directory
for scoredCSV in scoredCSVs:
    topicdate = Path(scoredCSV).stem

    logger.info("%s : Reading search results from PubMed CSV %s", topicdate, scoredCSV)

    scored_df = pd.read_csv(scoredCSV)
    # Add a column "search_source" containing the value of scoredCSV
    scored_df["search_source"] = Path(scoredCSV).stem

    # Update the counter with the values from the "journal" column
    pubmed_pubmed_journal_counter.update(scored_df["journal"].dropna())

    # Aggregate search_source for each journal
    for journal, sources in scored_df.groupby("journal")["search_source"]:
        if journal not in search_source_aggregated:
            search_source_aggregated[journal] = set()
        search_source_aggregated[journal].update(sources)

# Sort the counter by occurrences in descending order
sorted_journal_counts = pubmed_pubmed_journal_counter.most_common()

# Prepare the final dataframe
journal_counts_data = []
for journal, count in sorted_journal_counts:
    aggregated_sources = "|".join(sorted(search_source_aggregated[journal]))
    journal_counts_data.append((journal, count, aggregated_sources))

journal_counts_df = pd.DataFrame(journal_counts_data, columns=["Journal", "Count", "Search_Source"])

# Export the journal counter to a CSV file
journal_counts_df.to_csv(f"{CSV_DIR}scored_journal_counts.csv", index=False)

# Display the final dataframe
journal_counts_df

,Journal,Count,Search_Source
0,Ultrasound Obstet Gynecol,425,CDH-202303-topcited|CDH-202303-topdevelopmenta...
1,Prenat Diagn,379,CDH-202303-topcited|CDH-202303-topdevelopmenta...
2,J Pediatr Surg,357,CDH-202303-topcited|CDH-202303-topdevelopmenta...
3,Am J Obstet Gynecol,340,CDH-202303-topcited|CDH-202303-topdevelopmenta...
4,Fetal Diagn Ther,303,CDH-202303-topcited|CDH-202303-topdevelopmenta...
...,...,...,...
448,J Anat,1,NTD-202303-topdisruptive
449,Health Commun,1,NTD-202303-topdisruptive
450,Stem Cell Res,1,NTD-202303-topdisruptive
451,Exp Ther Med,1,NTD-202303-topdisruptive
